Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import re
import time
from dotenv import load_dotenv
import logging as log
# from matplotlib import pyplot as plt
# from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock, stock_data_setup
from systemmanager import SystemNode
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

### Stocks Data

In [3]:
ticker = ['MRNA']

client = OpenAI()

# file_stocks = stock_data_setup(client=client, ticker=ticker, type='price', dic_files=dic_files)
# file_cashflow = stock_data_setup(client=client, ticker=ticker, type='cash', dic_files=dic_files)
# file_income_stmt = stock_data_setup(client=client, ticker=ticker, type='income', dic_files=dic_files)
# stocks_news = stock_data_setup(client=client, ticker=ticker, type='news')

# Have to manually update the tool_resources because the file_id can change
# dic_assistants['fin_analyst']['tool_resources'] = {
# 	'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
# }

# with open('config/assistants.json', 'w') as json_file:
# 	json.dump(dic_assistants, json_file, indent='\t')
# 	print(f'assistants.json file has been updated')

# fin_analyst = AgentHandler(
# 	client = client, 
# 	new=False,
# 	assistant_name = 'fin_analyst',
# 	dic_file = dic_assistants
# 	)

# fin_consultant = AgentHandler(
# 	client = client,
# 	new=False,
# 	assistant_name = 'fin_consultant',
# 	dic_file = dic_assistants
# 	)

In [4]:
ag_ceo = AgentHandler(
	client = client, 
	new=False,
	assistant_name = 'ceo',
	dic_file = dic_assistants
	)

ag_market_research_manager = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'market_research_manager',
	dic_file = dic_assistants
	)

ag_industry_news_analyst = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'industry_news_analyst',
	dic_file = dic_assistants
	)

ag_company_news_analyst = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'company_news_analyst',
	dic_file = dic_assistants
	)

ag_press_release_analyst = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'press_release_analyst',
	dic_file = dic_assistants
	)

ag_stock_price_manager = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'stock_price_manager',
	dic_file = dic_assistants
	)

ag_stock_price_analyst = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'stock_price_analyst',
	dic_file = dic_assistants
	)


ag_financial_data_manager = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'financial_data_manager',
	dic_file = dic_assistants
	)

ag_income_statement_analyst = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'income_statement_analyst',
	dic_file = dic_assistants
	)

ag_balance_sheet_analyst = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'balance_sheet_analyst',
	dic_file = dic_assistants
	)

ag_cash_flow_analyst = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'cash_flow_analyst',
	dic_file = dic_assistants
	)

Assistant has been updated, name: ceo, id: asst_wvpcZU5aOJUalB1LXAqnOuyt
Assistant has been updated, name: market_research_manager, id: asst_85Pyx79Nbjg4GS2df2PFKmja
Assistant has been updated, name: industry_news_analyst, id: asst_tUtERZ4G8hR2a0vsgmvecDxE
Assistant has been updated, name: company_news_analyst, id: asst_ZMTZ8KLdnwp1L06TRPRIk7b0
Assistant has been updated, name: press_release_analyst, id: asst_cIorJMUDCSkafpRXXKLfhC3A
Assistant has been updated, name: stock_price_manager, id: asst_Smga0Hhjx5zcYHPCSCTDK0F1
Assistant has been updated, name: stock_price_analyst, id: asst_vmQ4udsZsVAgJv4OEXYxj72h
Assistant has been updated, name: financial_data_manager, id: asst_z0EwtpwaTg2VIzd0puFaF5Ax
Assistant has been updated, name: income_statement_analyst, id: asst_WnNOv9IIsKPbLCVeiIzm5rMj
Assistant has been updated, name: balance_sheet_analyst, id: asst_Rpp16fFsU4bgEr3WBb4kMe0L
Assistant has been updated, name: cash_flow_analyst, id: asst_3HIEYeBkm6xXlHgPy2PdVK2n


In [5]:
nd_market_research = SystemNode(client=client, name='market_research', agents=[ag_market_research_manager, ag_industry_news_analyst, ag_company_news_analyst, ag_press_release_analyst])

nd_stock_price = SystemNode(client=client, name='stock_price', agents=[ag_stock_price_manager, ag_stock_price_analyst])

nd_financial_data = SystemNode(client=client, name='financial_data', agents=[ag_stock_price_manager, ag_stock_price_analyst])

nd_management = SystemNode(client=client, name='management', agents=[ag_ceo, ag_market_research_manager, ag_stock_price_manager, ag_financial_data_manager], child_nodes=[nd_market_research, nd_stock_price, nd_financial_data])

In [ ]:
sample_prompt = f"I want your analysis on the following stock: {ticker}.\
	You are provided with the historical stock price data, and the following news article:\
	{stocks_news['MRNA'][0]}"

In [28]:
yfin = yf.Tickers('MSFT')

yfin.tickers['MSFT'].history(period='1')

MSFT: Period '1' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [4]:
prompt_start = f"This is the financial consultant.\
	The client wants advice on the following stocks.\
	I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
	You will be provided with the data of these stocks in .csv format. Use them as you see fit.\
	Stocks: {ticker}"

thread = ThreadManager(
	client=client,
	prompt=prompt_start
)

thread.run_thread(
	assistant=fin_analyst
	# prompt=prompt_start,
	# attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
)

next_prompt = thread.last_message

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > code_interpreter

import pandas as pd

# Load the data from the uploaded CSV file
file_path = '/mnt/data/file-jWdAWCIZ5J7BJ99WXAdcsIP5'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
data.head()
assistant > Here are the information gathered from the financial analyst: 

These are the stocks the client wants advice on: 
- Moderna, Inc. (MRNA)

Based on the provided data, here is an analysis of Moderna, Inc. (MRNA):

### Historical Price Analysis (Duration: December 2018 to Present)

1. **Price Trends**:
   - **Initial Opening Price**: $22.00 (Dec 7, 2018)
   - **Recent Closing Price**: $18.76 (Dec 13, 2018)
   - **Historical High and Low**:
     - **Highest Price**: This can be analyzed by looking at the "High" column across all records.
     - **Lowest Price**: This can be analyzed by looking at the "Low" column across all records.

2. **Volatility**:
   - Significant fluctuations were observed within the initial da

In [5]:
next_prompt = 'This message is for testing. Can you give a short sample response.'

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)

print(thread.last_message)


assistant > Sure, here's a short sample response:

---

### Recommendation for Moderna, Inc. (MRNA)

**Strengths**:
1. **High Growth Potential**: Strong involvement in mRNA technology.
2. **Significant Market Interest**: High trading volumes indicate investor confidence.

**Weaknesses**:
1. **Volatility**: Significant price fluctuations present higher risk.
2. **No Dividends**: Not suitable for income-focused investors.

**Fit with Strategy**:
- **Growth-Oriented Investors**: Suitable due to high growth potential.
- **Risk-Averse Investors**: Caution advised due to volatility.
- **Income-Focused Investors**: Not recommended since it doesn’t offer dividends.

### Final Recommendation:
Moderna, Inc. (MRNA) is recommended for growth-oriented investors with higher risk tolerance. Not ideal for conservative or income-focused strategies.

---

Please let me know if you need any further details!get_last_message initiated
asst_id = blank
message.assistant_id != asst_id, record previous messag

In [6]:
thread.df_messages

,message_id,assistant_id,created_at,file_ids,role,run_id,message_text,_msg_loc
0,msg_ix1bKCixCKmonpOUUcwEA1ma,None,1727333036,[],user,None,This is the financial consultant.\tThe client ...,0
1,msg_av7UCA1kgMGRZ01gCpv4Ury6,asst_Mqf9cO1sDTOd4UMYwcFfIQrA,1727333042,[],assistant,run_E0zS7pImG3gCfk5W0euE7UG2,Here are the information gathered from the fin...,1
2,msg_MV1Z4VnNef62FVL3JloElTIk,None,1727333054,[],user,None,Here are the information gathered from the fin...,2
3,msg_lSPZOxGoeUugyaV1yrokW6Ey,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1727333055,[],assistant,run_XvEl5sGLSsT7me9d5BFLU4d8,"### Recommendation for Moderna, Inc. (MRNA)\n\...",3
4,msg_YtXAmLTidZAAWuOaWWswUtLE,None,1727333074,[],user,None,This message is for testing. Can you give a sh...,4
5,msg_RVGjKzoJzEA545GR4csDTgKz,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1727333075,[],assistant,run_KHvUeX1f2U5WBrtgA2BwTDoc,"Sure, here's a short sample response:\n\n---\n...",5


In [35]:
thread.df_messages

,message_id,assistant_id,created_at,file_ids,role,run_id,message_text,_msg_loc
0,msg_jNKhekiTCZtm8w4FPvX15gAj,None,1726556528,[],user,None,This is the financial consultant.\tThe client ...,0
1,msg_ZnRQdMDKHoD1H5rtlXqHchrD,asst_Mqf9cO1sDTOd4UMYwcFfIQrA,1726556533,[],assistant,run_SUXZgjKFGQXWadJINBkRKVKT,Here are the information gathered from the fin...,1
2,msg_Uk5Nu4x58DtDkKMAq2hD1caL,None,1726556545,[],user,None,Here are the information gathered from the fin...,2
3,msg_OeTIVYGnagoJeGndj95UJJFG,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1726556546,[],assistant,run_e3Wu9olqlU7Ts7zIrWRpeAoY,### Recommendation for Moderna Inc. (MRNA)\n\n...,3
4,msg_2oqodrxs428y1uKMA8I38ikI,None,1726556574,[],user,None,This message is for testing. Can you give a sh...,4
5,msg_tXf7w2DLQf3jXx2Vihici8De,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1726556575,[],assistant,run_zhpjoi5bMscVc2i6fZWGL35t,### Recommendation for Moderna Inc. (MRNA)\n\n...,5


In [20]:
thread.delete_thread()

thread: thread_mPIjTp06Kua0Y9HZvV0eUdh3 has been deleted.


In [3]:
yfin = YFHandler(stock_list=['MSFT'],schemas=dic_files)

In [10]:
dic_links, dic_articles = yfin.get_stock_news(max_links=5)
# type(dic_links)

In [19]:
# print(dic_articles['MSFT'][0])
dic_links
print(dic_articles['MSFT'][2])

Lumen Technologies (NYSE: LUMN), the telecom company formerly known as CenturyLink, seemed to be in dire straits just a few years ago. Its revenue was declining, it was racking up steep losses, and it suspended its dividend in 2022.
But over the past 12 months, Lumen's stock soared 374% as a new AI infrastructure deal with Microsoft breathed fresh life into its business. Could those tailwinds propel its stock even higher over the next year?
Lumen is one of the largest wireline service providers in the United States. Unlike AT&T and Verizon, which expanded their wireless networks to reduce their dependence on wireline connections, Lumen shunned the wireless market and expanded its wireline business through a series of mergers and acquisitions.
Lumen expected to generate slow but steady growth as economies of scale kicked in. It also expanded its faster fiber networks and bundled more cloud, security, and collaboration services into its business wireline plans.
Lumen's smaller fiber busi